In [1]:
import argparse
from glob import glob
import tensorflow as tf
import time
from model import denoiser
import os
import numpy as np

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args = Namespace(epoch=10, batch_size=32, lr=0.001, use_gpu=1, phase='train',ckpt_dir='./checkpoint', test_dir='./data/denoised')


In [3]:
def denoiser_train(denoiser, lr):
        noisy_eval_files = glob('./data/train/noisy/*.png')
        noisy_eval_files = sorted(noisy_eval_files)
        eval_files = glob('./data/train/original/*.png')
        eval_files = sorted(eval_files)
        denoiser.train(eval_files, noisy_eval_files, batch_size=args.batch_size, ckpt_dir=args.ckpt_dir, epoch=args.epoch, lr=lr)



In [4]:

def denoiser_test(denoiser):

    noisy_eval_files = glob('./data/test/noisy/*.png')
#    n = [int(i) for i in map(lambda x: x.split('/')[-1].split('.png')[0], noisy_eval_files)]
#    noisy_eval_files = [x for (y, x) in sorted(zip(n, noisy_eval_files))]
    noisy_eval_files = sorted(noisy_eval_files)
    eval_files = glob('./data/test/original/*.png')
    eval_files = sorted(eval_files)
    start = time.time()
    denoiser.test(eval_files, noisy_eval_files, ckpt_dir=args.ckpt_dir, save_dir='./data/denoised')
    end = time.time()
    print "Elapsed time:", end-start

In [5]:
def run():
    if not os.path.exists(args.ckpt_dir):
        os.makedirs(args.ckpt_dir)
    if not os.path.exists(args.test_dir):
        os.makedirs(args.test_dir)

    lr = args.lr * np.ones([args.epoch])
    lr[30:] = lr[0] / 10.0
    if args.use_gpu:
        # added to control the gpu memory
        print("GPU\n")
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
            model = denoiser(sess)
            if args.phase == 'train':
                denoiser_train(model, lr=lr)
            elif args.phase == 'test':
                denoiser_test(model)
            else:
                print('[!]Unknown phase')
                exit(0)
    else:
        print("CPU\n")
        with tf.Session() as sess:
            model = denoiser(sess)
            if args.phase == 'train':
                denoiser_train(model, lr=lr)
            elif args.phase == 'test':
                denoiser_test(model)
            else:
                print('[!]Unknown phase')
                exit(0)


In [6]:



def test():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
      model = denoiser(sess)
      denoiser_test(model)

In [ ]:
run()

GPU

[*] Initialize model successfully...
[*] Reading checkpoint...
[*] Not find pretrained model!
